In [5]:
import cleaning

In [3]:
cleaned_trr = cleaning.process_trr()

In [6]:
cleaned_trr

,id,event_number,cr_number,beat,block,direction,street,location,trr_datetime,indoor_or_outdoor,...,officer_race,officer_age,officer_appointed_date,officer_birth_year,officer_unit_name,officer_unit_detail,trr_created,latitude,longitude,point
0,96630,1611411724,None,1124,30XX,West,Jackson Blvd,Street,2016-04-23 18:18:00-05:00,None,...,WHITE,30,2014-02-18,None,11,None,None,41.8776497,-87.7012653,0101000020E61000007554DA87E1EC55C00F6C4BD356F0...
1,97999,1618611539,None,723,9XX,West,Marquette Rd,Street,2016-07-04 17:06:00-05:00,Outdoor,...,WHITE,30,2013-11-25,None,7,None,None,41.7724444,-87.6470302,0101000020E61000008F215BF168E955C03AFD4575DFE2...
2,98796,1623610044,None,811,53XX,South,Narragansett Ave,Residence,2016-08-23 16:14:00-05:00,Outdoor,...,WHITE,51,2002-09-30,None,8,None,None,41.7959057,-87.7821939,0101000020E61000005DE800770FF255C09919EC3CE0E5...
3,99293,1626900073,None,2232,100XX,South,Emerald Ave,Sidewalk,2016-09-25 20:00:00-05:00,None,...,WHITE,34,2009-12-16,None,22,None,None,41.7122367,-87.6421715,0101000020E610000017BB7D5619E955C067C17A922ADB...
4,99369,1627409687,None,634,XX,West,95Th St,Sidewalk,2016-09-30 17:13:00-05:00,None,...,BLACK,46,2006-02-21,None,6,None,None,41.72199,-87.626064,0101000020E6100000FD4CBD6E11E855C0FF04172B6ADC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17460,6043642,262,None,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,HISPANIC,-25,2019-02-19,1995,25,None,2020-05-26 03:42:00-05:00,41.9027893,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...
17461,6043651,262,None,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,HISPANIC,-24,2019-08-16,1996,25,None,2020-05-26 03:45:00-05:00,41.9027893,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...
17462,6059341,2014902656,None,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,HISPANIC,-31,2017-09-18,1989,10,None,2020-05-28 06:52:00-05:00,41.8540554,-87.6908109,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...
17463,6059467,2014902656,None,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,HISPANIC,-26,2018-11-27,1994,10,None,2020-05-28 07:24:00-05:00,41.8540554,-87.6908109,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...
